# Todo o relatório deste MVP consta anexado no diretório do Github.

###Carga de arquivos CSV

Comando que cria diretório no DBFS

In [ ]:
dbutils.fs.mkdirs('/FileStore/tables/Orders')

Out[1]: True

In [ ]:
display(dbutils.fs.ls('dbfs:/FileStore/tables/Orders'))

path,name,size,modificationTime
dbfs:/FileStore/tables/Orders/olist_customers_dataset.csv,olist_customers_dataset.csv,9033957,1742396044000
dbfs:/FileStore/tables/Orders/olist_geolocation_dataset.csv,olist_geolocation_dataset.csv,61273883,1742396058000
dbfs:/FileStore/tables/Orders/olist_order_items_dataset.csv,olist_order_items_dataset.csv,15438671,1742396049000
dbfs:/FileStore/tables/Orders/olist_order_payments_dataset.csv,olist_order_payments_dataset.csv,5777138,1742396050000
dbfs:/FileStore/tables/Orders/olist_order_reviews_dataset.csv,olist_order_reviews_dataset.csv,14451670,1742396055000
dbfs:/FileStore/tables/Orders/olist_orders_dataset.csv,olist_orders_dataset.csv,17654914,1742396060000
dbfs:/FileStore/tables/Orders/olist_products_dataset.csv,olist_products_dataset.csv,2379446,1742396059000
dbfs:/FileStore/tables/Orders/olist_sellers_dataset.csv,olist_sellers_dataset.csv,174703,1742396060000
dbfs:/FileStore/tables/Orders/product_category_name_translation.csv,product_category_name_translation.csv,2613,1742396060000


Criando banco de dados no sistema de gerenciamento de dados - Camada **Bronze**

In [ ]:
%sql
CREATE DATABASE IF NOT EXISTS Bronze;

Criando tabelas no banco Bronze, que lêem diretamente os dados dos CSVs localizados no caminho especificado


In [ ]:
%sql

use Bronze;
CREATE table IF NOT EXISTS FatoPedidos
USING csv OPTIONS ('path' '/FileStore/tables/Orders/olist_orders_dataset.csv', 'header' 'true');

CREATE table IF NOT EXISTS DimReviews_Pedidos
USING csv OPTIONS ('path' '/FileStore/tables/Orders/olist_order_reviews_dataset.csv', 'header' 'true');

CREATE table IF NOT EXISTS DimClientes
USING csv OPTIONS ('path' '/FileStore/tables/Orders/olist_customers_dataset.csv', 'header' 'true');

CREATE table IF NOT EXISTS DimGeolocalizacao
USING csv OPTIONS ('path' '/FileStore/tables/Orders/olist_geolocation_dataset.csv', 'header' 'true');

CREATE table IF NOT EXISTS DimItens_Pedidos
USING csv OPTIONS ('path' '/FileStore/tables/Orders/olist_order_items_dataset.csv', 'header' 'true');

CREATE table IF NOT EXISTS DimPagamentos_Pedidos
USING csv OPTIONS ('path' '/FileStore/tables/Orders/olist_order_payments_dataset.csv', 'header' 'true');

CREATE table IF NOT EXISTS DimProdutos
USING csv OPTIONS ('path' '/FileStore/tables/Orders/olist_products_dataset.csv', 'header' 'true');

CREATE table IF NOT EXISTS DimVendedores
USING csv OPTIONS ('path' '/FileStore/tables/Orders/olist_sellers_dataset.csv', 'header' 'true');

In [ ]:
%sql SELECT * FROM Bronze.FatoPedidos LIMIT 5

order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


Criando banco de dados no sistema de gerenciamento de dados - Camada **Silver**

In [ ]:
%sql
CREATE DATABASE IF NOT EXISTS Silver;

In [ ]:
clientes_df = spark.read.format("csv").option("header", "true").option("inferschema", "true").load("dbfs:/FileStore/tables/Orders/olist_customers_dataset.csv")
localizacao_df = spark.read.format("csv").option("header", "true").option("inferschema", "true").load("dbfs:/FileStore/tables/Orders/olist_geolocation_dataset.csv")
itens_df = spark.read.format("csv").option("header", "true").option("inferschema", "true").load("dbfs:/FileStore/tables/Orders/olist_order_items_dataset.csv")
pagamentos_df = spark.read.format("csv").option("header", "true").option("inferschema", "true").load("dbfs:/FileStore/tables/Orders/olist_order_payments_dataset.csv")
reviews_df = spark.read.format("csv").option("header", "true").option("inferschema", "true").load("dbfs:/FileStore/tables/Orders/olist_order_reviews_dataset.csv")
pedidos_df = spark.read.format("csv").option("header", "true").option("inferschema", "true").load("dbfs:/FileStore/tables/Orders/olist_orders_dataset.csv")
produtos_df = spark.read.format("csv").option("header", "true").option("inferschema", "true").load("dbfs:/FileStore/tables/Orders/olist_products_dataset.csv")
vendedores_df = spark.read.format("csv").option("header", "true").option("inferschema", "true").load("dbfs:/FileStore/tables/Orders/olist_sellers_dataset.csv")

In [ ]:
clientes_df.write.format("delta").mode("overwrite").saveAsTable("Silver.DimClientes")
localizacao_df.write.format("delta").mode("overwrite").saveAsTable("Silver.DimLocalizacao")
itens_df.write.format("delta").mode("overwrite").saveAsTable("Silver.DimItens")
pagamentos_df.write.format("delta").mode("overwrite").saveAsTable("Silver.DimPagamentos")
reviews_df.write.format("delta").mode("overwrite").saveAsTable("Silver.DimReviews")
pedidos_df.write.format("delta").mode("overwrite").saveAsTable("Silver.FatoPedidos")
produtos_df.write.format("delta").mode("overwrite").saveAsTable("Silver.DimProdutos")
vendedores_df.write.format("delta").mode("overwrite").saveAsTable("Silver.DimVendedores")

In [ ]:
%sql SELECT * FROM Silver.FatoPedidos LIMIT 5

order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,payment_total,total_items
e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02T10:56:33.000+0000,2017-10-02T11:07:15.000+0000,2017-10-04T19:55:00.000+0000,2017-10-10T21:25:13.000+0000,2017-10-18T00:00:00.000+0000,null,null
53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24T20:41:37.000+0000,2018-07-26T03:24:27.000+0000,2018-07-26T14:31:00.000+0000,2018-08-07T15:27:45.000+0000,2018-08-13T00:00:00.000+0000,null,null
47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08T08:38:49.000+0000,2018-08-08T08:55:23.000+0000,2018-08-08T13:50:00.000+0000,2018-08-17T18:06:29.000+0000,2018-09-04T00:00:00.000+0000,null,null
949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18T19:28:06.000+0000,2017-11-18T19:45:59.000+0000,2017-11-22T13:39:59.000+0000,2017-12-02T00:28:42.000+0000,2017-12-15T00:00:00.000+0000,null,null
ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13T21:18:39.000+0000,2018-02-13T22:20:29.000+0000,2018-02-14T19:46:34.000+0000,2018-02-16T18:17:02.000+0000,2018-02-26T00:00:00.000+0000,null,null


Visualizando informações sobre as tabelas

In [ ]:
%sql
DESCRIBE silver.fatopedidos

col_name,data_type,comment
order_id,string,null
customer_id,string,null
order_status,string,null
order_purchase_timestamp,timestamp,null
order_approved_at,timestamp,null
order_delivered_carrier_date,timestamp,null
order_delivered_customer_date,timestamp,null
order_estimated_delivery_date,timestamp,null
payment_total,"decimal(10,2)",null
total_items,int,null


In [ ]:
%sql
DESCRIBE silver.dimclientes

col_name,data_type,comment
customer_id,string,null
customer_unique_id,string,null
customer_zip_code_prefix,int,null
customer_city,string,null
customer_state,string,null


In [ ]:
%sql
DESCRIBE silver.dimlocalizacao

col_name,data_type,comment
geolocation_zip_code_prefix,int,null
geolocation_lat,double,null
geolocation_lng,double,null
geolocation_city,string,null
geolocation_state,string,null


In [ ]:
%sql
DESCRIBE silver.dimitens

col_name,data_type,comment
order_id,string,null
order_item_id,int,null
product_id,string,null
seller_id,string,null
shipping_limit_date,timestamp,null
price,double,null
freight_value,double,null


In [ ]:
%sql
DESCRIBE silver.dimreviews

col_name,data_type,comment
review_id,string,null
order_id,string,null
review_score,string,null
review_comment_title,string,null
review_comment_message,string,null
review_creation_date,string,null
review_answer_timestamp,string,null


In [ ]:
%sql
DESCRIBE silver.dimprodutos

col_name,data_type,comment
product_id,string,null
product_category_name,string,null
product_name_lenght,int,null
product_description_lenght,int,null
product_photos_qty,int,null
product_weight_g,int,null
product_length_cm,int,null
product_height_cm,int,null
product_width_cm,int,null


In [ ]:
%sql
DESCRIBE silver.dimvendedores

col_name,data_type,comment
seller_id,string,null
seller_zip_code_prefix,int,null
seller_city,string,null
seller_state,string,null


In [ ]:
%sql
DESCRIBE silver.dimpagamentos

col_name,data_type,comment
order_id,string,null
payment_sequential,int,null
payment_type,string,null
payment_installments,int,null
payment_value,double,null


##Transformação - Camada Silver

Após alguns testes, criarei novas colunas para a tabela fato

In [ ]:
%sql
CREATE OR REPLACE TABLE Silver.fatopedidos_teste
USING DELTA
AS
SELECT
    order_id,
    customer_id,
    order_status,
    order_purchase_timestamp,
    order_approved_at,
    order_delivered_carrier_date,
    order_delivered_customer_date,
    order_estimated_delivery_date
FROM Silver.fatopedidos;

num_affected_rows,num_inserted_rows


Adicionando a coluna de valor total dos itens do pedido

In [ ]:
%sql
ALTER TABLE Silver.fatopedidos_teste ADD COLUMN total_order_item_value DECIMAL(10,2);

Valor total para cada pedido

In [ ]:
%sql
MERGE INTO Silver.fatopedidos_teste AS f
USING (
    SELECT order_id, SUM(price) AS total_order_item_value
    FROM Silver.DimItens
    GROUP BY order_id
) AS i
ON f.order_id = i.order_id
WHEN MATCHED THEN
    UPDATE SET f.total_order_item_value = i.total_order_item_value;

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
98666,98666,0,0


Adicionando a coluna de valor total do frete à tabela fato

In [ ]:
%sql
ALTER TABLE Silver.fatopedidos_teste ADD COLUMN total_freight_value DECIMAL(10,2);

In [ ]:
%sql
MERGE INTO Silver.fatopedidos_teste AS f
USING (
    SELECT order_id, SUM(freight_value) AS total_freight_value
    FROM Silver.DimItens
    GROUP BY order_id
) AS i
ON f.order_id = i.order_id
WHEN MATCHED THEN
    UPDATE SET f.total_freight_value = i.total_freight_value;

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
98666,98666,0,0


Adicionando a coluna de valor total do pedido

In [ ]:
%sql
ALTER TABLE Silver.fatopedidos_teste ADD COLUMN total_order_value  DECIMAL(10,2);

In [ ]:
%sql
UPDATE Silver.fatopedidos_teste
SET total_order_value = total_order_item_value + total_freight_value;

num_affected_rows
99441


Adicionando coluna de total de itens por pedido à tabela fato


In [ ]:
%sql
ALTER TABLE Silver.fatopedidos_teste ADD COLUMN total_items INT;

Número de itens para cada order_id

Número de itens para cada order_id

In [ ]:
%sql
MERGE INTO Silver.fatopedidos_teste AS f
USING (
    SELECT i.order_id, COUNT(i.order_item_id) AS total_items
    FROM Silver.DimItens i
    GROUP BY i.order_id
) AS items
ON f.order_id = items.order_id
WHEN MATCHED THEN
    UPDATE SET f.total_items = items.total_items;

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
98666,98666,0,0


In [ ]:
%sql
DESCRIBE Silver.fatopedidos_teste;

col_name,data_type,comment
order_id,string,null
customer_id,string,null
order_status,string,null
order_purchase_timestamp,timestamp,null
order_approved_at,timestamp,null
order_delivered_carrier_date,timestamp,null
order_delivered_customer_date,timestamp,null
order_estimated_delivery_date,timestamp,null
total_order_item_value,"decimal(10,2)",null
total_freight_value,"decimal(10,2)",null


Verificando se há order_id na tabela DimItens que não estão presentes na tabela fato

In [ ]:
%sql
SELECT DISTINCT i.order_id
FROM Silver.DimItens i
LEFT JOIN Silver.fatopedidos_teste f
  ON i.order_id = f.order_id
WHERE f.order_id IS NULL;

order_id


Verificando se há order_id na tabela fato que não estão presentes na tabela DimItens

In [ ]:
%sql
SELECT DISTINCT f.order_id
FROM Silver.fatopedidos_teste f
LEFT JOIN Silver.DimItens i
  ON f.order_id = i.order_id
WHERE i.order_id IS NULL
LIMIT 15;

order_id
c272bcd21c287498b4883c7512019702
7622da48f027d5659a0d7b0a43597354
f0f94b7c7548150f33f5d9e7597d396f
23fdd0e16ad79ceab4e5f407772d64fa
410d704bdcbe2893fd5ad73fe8166f43
b2673ed6360dc341a4728dbd632f935b
5ae7a6fa175ab7b180b6c0142e5466e9
03d1edbd314ca7682ec0f3e67d3763e2
f5e0327c97a90df59977a815f6e4453c
752c0993c4993edea8eac33fda648244


Verificando possíveis motivos para order_id ausentes na tabela DimItens

In [ ]:
%sql
SELECT f.order_id, f.customer_id, f.order_status
FROM Silver.fatopedidos_teste f
LEFT JOIN Silver.DimItens i ON f.order_id = i.order_id
WHERE i.order_id IS NULL
LIMIT 15;

order_id,customer_id,order_status
c272bcd21c287498b4883c7512019702,9582c5bbecc65eb568e2c1d839b5cba1,unavailable
7622da48f027d5659a0d7b0a43597354,b73059b796c2abebd783eb6af533b1a2,unavailable
f0f94b7c7548150f33f5d9e7597d396f,cb5a3d3dea83e4200af551141613ddab,unavailable
23fdd0e16ad79ceab4e5f407772d64fa,f915ff705480d8952c4e91371c39e95e,unavailable
410d704bdcbe2893fd5ad73fe8166f43,0424e2f81bdfb77d7c11c3f447c7b241,unavailable
b2673ed6360dc341a4728dbd632f935b,acd30e8ac809a440f8a10918d0683a17,unavailable
5ae7a6fa175ab7b180b6c0142e5466e9,bc7b861ebaf4366268881e14b4a52666,unavailable
03d1edbd314ca7682ec0f3e67d3763e2,963bc458a6c7f3148e1a756a4fc1e012,unavailable
f5e0327c97a90df59977a815f6e4453c,e6c59c09cb529c3f49bf5aa8e345d2e8,unavailable
752c0993c4993edea8eac33fda648244,a1bf5849a8b9f1535ddce758f6c0c41e,unavailable


Aparentemente, todos os casos de order_id que não estão presentes na tabela de itens possuem status 'unavailable', 'canceled', 'unvoiced' ou 'created', ou seja, nenhum 'delivered'. Isso indica que estes pedidos podem ter um comportamento diferente em relação à inclusão de itens na tabela DimItens. Isto sugere que estes status específicos podem não ter sido tratados corretamente ou não foram considerados relevantes para o preenchimento da tabela DimItens.

In [ ]:
%sql
SELECT DISTINCT f.order_status
FROM Silver.fatopedidos_teste f
LEFT JOIN Silver.DimItens i ON f.order_id = i.order_id
WHERE i.order_id IS NULL;

order_status
shipped
canceled
created
unavailable
invoiced


Como qualquer sistema de e-commerce, é possível  verificar que um pedido pode ter de 1 a x itens.

In [ ]:
%sql
SELECT order_id, COUNT(*) AS num_itens
FROM Silver.DimItens
GROUP BY order_id
ORDER BY num_itens DESC
LIMIT 15;

order_id,num_itens
8272b63d03f5f79c56e9e4120aec44ef,21
1b15974a0141d54e36626dca3fdc731a,20
ab14fdcfbe524636d65ee38360e22ce8,20
428a2f660dc84138d969ccd69a0ab6d5,15
9ef13efd6949e4573a18964dd1bbe7f5,15
73c8ab38f07dc94389065f7eba4f297a,14
9bdc4d4c71aa1de4606060929dee888c,14
37ee401157a3a0b28c9c6d0ed8c3b24b,13
c05d6a79e55da72ca780ce90364abed9,12
af822dacd6f5cff7376413c03a388bb7,12


Verificando se há pedidos duplicados na tabela fato, com base na coluna order_id

In [ ]:
%sql
SELECT order_id, COUNT(*)
FROM silver.fatopedidos_teste
GROUP BY order_id
HAVING COUNT(*) > 1;

order_id,count(1)


Integridade referencial entre a tabela fato e DimClientes

In [ ]:
%sql
SELECT f.order_id, f.customer_id
FROM Silver.fatopedidos_teste f
LEFT JOIN Silver.DimClientes c
  ON f.customer_id = c.customer_id
WHERE c.customer_id IS NULL; -- verificando que cada customer_id na tabela fato tenha uma correspondência válida na tabela DimClientes

order_id,customer_id


Verificando que é preciso transformar os tipos de dados de algumas colunas de DimItens.

In [ ]:
%sql
DESCRIBE Silver.dimitens

col_name,data_type,comment
order_id,string,null
order_item_id,int,null
product_id,string,null
seller_id,string,null
shipping_limit_date,timestamp,null
price,double,null
freight_value,double,null


In [ ]:
%sql
CREATE OR REPLACE TABLE Silver.dimitens_final
USING DELTA
AS
SELECT
    order_id,
    order_item_id,
    product_id,
    seller_id,
    shipping_limit_date,
    CAST(price AS DECIMAL(10,2)) AS price,
    CAST(freight_value AS DECIMAL(10,2)) AS freight_value
FROM Silver.dimitens;

num_affected_rows,num_inserted_rows


Verificando tipos de dados das tabelas

In [ ]:
%sql
DESCRIBE Silver.dimclientes;

col_name,data_type,comment
customer_id,string,null
customer_unique_id,string,null
customer_zip_code_prefix,int,null
customer_city,string,null
customer_state,string,null


In [ ]:
%sql
DESCRIBE Silver.dimlocalizacao;

col_name,data_type,comment
geolocation_zip_code_prefix,int,null
geolocation_lat,double,null
geolocation_lng,double,null
geolocation_city,string,null
geolocation_state,string,null


In [ ]:
%sql
DESCRIBE Silver.dimpagamentos;


col_name,data_type,comment
order_id,string,null
payment_sequential,int,null
payment_type,string,null
payment_installments,int,null
payment_value,double,null


Alterando o tipo de dado da coluna payment_value da tabela DimPagamentos para Decimal.

In [ ]:
%sql
CREATE OR REPLACE TABLE Silver.dimpagamentos_final
USING DELTA
AS
SELECT
    order_id,
    payment_sequential,
    payment_type,
    payment_installments,
    CAST(payment_value AS DECIMAL(10,2)) AS payment_value
FROM Silver.dimpagamentos;

num_affected_rows,num_inserted_rows


In [ ]:
%sql
DESCRIBE Silver.dimreviews

col_name,data_type,comment
review_id,string,null
order_id,string,null
review_score,string,null
review_comment_title,string,null
review_comment_message,string,null
review_creation_date,string,null
review_answer_timestamp,string,null


Alterando coluna review_score para o tipo de dado inteiro.

In [ ]:
%sql
CREATE OR REPLACE TABLE Silver.dimreviews_final
USING DELTA
AS
SELECT
  review_id,
  order_id,
  CAST(review_score AS INT) AS review_score,
  review_comment_title,
  review_comment_message,
  review_creation_date,
  review_answer_timestamp
FROM Silver.dimreviews;

num_affected_rows,num_inserted_rows


In [ ]:
%sql
DESCRIBE Silver.dimreviews_final

col_name,data_type,comment
review_id,string,null
order_id,string,null
review_score,int,null
review_comment_title,string,null
review_comment_message,string,null
review_creation_date,string,null
review_answer_timestamp,string,null


In [ ]:
%sql
DESCRIBE Silver.dimprodutos

col_name,data_type,comment
product_id,string,null
product_category_name,string,null
product_name_lenght,int,null
product_description_lenght,int,null
product_photos_qty,int,null
product_weight_g,int,null
product_length_cm,int,null
product_height_cm,int,null
product_width_cm,int,null


In [ ]:
%sql
DESCRIBE Silver.dimvendedores;

col_name,data_type,comment
seller_id,string,null
seller_zip_code_prefix,int,null
seller_city,string,null
seller_state,string,null


In [ ]:
from pyspark.sql.functions import date_format

Formatando colunas de data para formato específico.

In [ ]:
%sql
CREATE OR REPLACE TABLE Silver.fatopedidos_new -- criando mais uma cópia da tabela fato
USING DELTA
AS
SELECT
  order_id,
  customer_id,
  order_status,
  date_format(order_purchase_timestamp, 'yyyy-MM-dd HH:mm:ss') AS order_purchase_timestamp_formatted,
  date_format(order_approved_at, 'yyyy-MM-dd HH:mm:ss') AS order_approved_at_formatted,
  date_format(order_delivered_carrier_date, 'yyyy-MM-dd HH:mm:ss') AS oorder_delivered_carrier_date_formatted,
  date_format(order_delivered_customer_date, 'yyyy-MM-dd HH:mm:ss') AS order_delivered_customer_date_formatted,
  date_format(order_estimated_delivery_date, 'yyyy-MM-dd HH:mm:ss') AS order_estimated_delivery_date_formatted,
  total_order_item_value,
  total_freight_value,
  total_order_value,
  total_items
FROM Silver.fatopedidos_teste;

num_affected_rows,num_inserted_rows


In [ ]:
%sql SELECT * FROM Silver.fatopedidos_new LIMIT 5

order_id,customer_id,order_status,order_purchase_timestamp_formatted,order_approved_at_formatted,oorder_delivered_carrier_date_formatted,order_delivered_customer_date_formatted,order_estimated_delivery_date_formatted,total_order_item_value,total_freight_value,total_order_value,total_items
8e24261a7e58791d10cb1bf9da94df5c,64a254d30eed42cd0e6c36dddb88adf0,unavailable,2017-11-16 15:09:28,2017-11-16 15:26:57,null,null,2017-12-05 00:00:00,null,null,null,null
c272bcd21c287498b4883c7512019702,9582c5bbecc65eb568e2c1d839b5cba1,unavailable,2018-01-31 11:31:37,2018-01-31 14:23:50,null,null,2018-02-16 00:00:00,null,null,null,null
37553832a3a89c9b2db59701c357ca67,7607cd563696c27ede287e515812d528,unavailable,2017-08-14 17:38:02,2017-08-17 00:15:18,null,null,2017-09-05 00:00:00,null,null,null,null
d57e15fb07fd180f06ab3926b39edcd2,470b93b3f1cde85550fc74cd3a476c78,unavailable,2018-01-08 19:39:03,2018-01-09 07:26:08,null,null,2018-02-06 00:00:00,null,null,null,null
00b1cb0320190ca0daa2c88b35206009,3532ba38a3fd242259a514ac2b6ae6b6,canceled,2018-08-28 15:26:39,null,null,null,2018-09-12 00:00:00,null,null,null,null


Verificando que as colunas de data não possuem mais as informações sobre o fuso horário, mas constam como strings.

In [ ]:
%sql
DESCRIBE Silver.fatopedidos_new

col_name,data_type,comment
order_id,string,null
customer_id,string,null
order_status,string,null
order_purchase_timestamp_formatted,string,null
order_approved_at_formatted,string,null
oorder_delivered_carrier_date_formatted,string,null
order_delivered_customer_date_formatted,string,null
order_estimated_delivery_date_formatted,string,null
total_order_item_value,"decimal(10,2)",null
total_freight_value,"decimal(10,2)",null


Adicionando novas colunas com tipo Date

In [ ]:
%sql
ALTER TABLE Silver.fatopedidos_new
ADD COLUMNS (
    order_purchase_timestamp_new DATE,
    order_approved_at_new DATE,
    order_delivered_carrier_date_new DATE,
    order_delivered_customer_date_new DATE,
    order_estimated_delivery_date_new DATE
);

Alterando colunas para o tipo Date.

In [ ]:
%sql
MERGE INTO Silver.fatopedidos_new AS target
USING (
    SELECT
        order_id,
        CAST(order_purchase_timestamp_formatted AS DATE) AS order_purchase_timestamp_new,
        CAST(order_approved_at_formatted AS DATE) AS order_approved_at_new,
        CAST(oorder_delivered_carrier_date_formatted AS DATE) AS order_delivered_carrier_date_new,
        CAST(order_delivered_customer_date_formatted AS DATE) AS order_delivered_customer_date_new,
        CAST(order_estimated_delivery_date_formatted AS DATE) AS order_estimated_delivery_date_new
    FROM Silver.fatopedidos_new
) AS source
ON target.order_id = source.order_id
WHEN MATCHED THEN
    UPDATE SET
        target.order_purchase_timestamp_new = source.order_purchase_timestamp_new,
        target.order_approved_at_new = source.order_approved_at_new,
        target.order_delivered_carrier_date_new = source.order_delivered_carrier_date_new,
        target.order_delivered_customer_date_new = source.order_delivered_customer_date_new,
        target.order_estimated_delivery_date_new = source.order_estimated_delivery_date_new;

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
99441,99441,0,0


Criando a tabela fato final

In [ ]:
%sql
CREATE OR REPLACE TABLE Silver.fatopedidos_final
AS
SELECT
    order_id,
    customer_id,
    order_status,
    total_items,
    total_order_item_value,
    total_freight_value,
    total_order_value,
    CAST(order_purchase_timestamp_formatted AS DATE) AS order_purchase_timestamp_new,
    CAST(order_approved_at_formatted AS DATE) AS order_approved_at_new,
    CAST(oorder_delivered_carrier_date_formatted AS DATE) AS order_delivered_carrier_date_new,
    CAST(order_delivered_customer_date_formatted AS DATE) AS order_delivered_customer_date_new,
    CAST(order_estimated_delivery_date_formatted AS DATE) AS order_estimated_delivery_date_new
FROM Silver.fatopedidos_new;

num_affected_rows,num_inserted_rows


In [ ]:
%sql
DESCRIBE Silver.fatopedidos_final

col_name,data_type,comment
order_id,string,null
customer_id,string,null
order_status,string,null
total_items,int,null
total_order_item_value,"decimal(10,2)",null
total_freight_value,"decimal(10,2)",null
total_order_value,"decimal(10,2)",null
order_purchase_timestamp_new,date,null
order_approved_at_new,date,null
order_delivered_carrier_date_new,date,null


In [ ]:
%sql SELECT * FROM Silver.fatopedidos_final LIMIT 15;

order_id,customer_id,order_status,total_items,total_order_item_value,total_freight_value,total_order_value,order_purchase_timestamp_new,order_approved_at_new,order_delivered_carrier_date_new,order_delivered_customer_date_new,order_estimated_delivery_date_new
b513b10ef81dc22d15e850789a8a96b1,fa3c9c28979e812523cac4180cf5bbca,delivered,1,179.00,18.53,197.53,2018-02-25,2018-02-27,2018-02-27,2018-03-08,2018-03-21
2d58f989c5e9344556ea2a9526699249,1fc09040150013ea0ef014f02dcb4e97,delivered,1,89.99,16.39,106.38,2017-10-01,2017-10-01,2017-10-04,2017-10-10,2017-10-27
a92225110bfd8206a036f272ccf0fb1b,7f96cdc06aa0cfe77e71187d5035f22f,delivered,1,49.00,18.23,67.23,2018-05-03,2018-05-03,2018-05-04,2018-05-15,2018-06-04
d17a342bb9f94d40c3b6df4a600b5cee,511dfb4f5249ddae8cec62d7c650859c,delivered,2,59.98,14.78,74.76,2018-04-06,2018-04-06,2018-04-09,2018-04-10,2018-04-24
8dd9758206f8d9c23baf77b965e1c6dc,849d4f35271a07987c6af0c676c5ca31,delivered,1,84.99,15.35,100.34,2017-11-28,2017-11-28,2017-12-04,2017-12-19,2017-12-20
7c4281b08578a1e5e13e56814b3c6123,686ba249f6f98cc2610b58e5b9982e6a,delivered,1,157.00,18.00,175.00,2017-07-18,2017-07-18,2017-07-19,2017-07-24,2017-08-24
2b9c1a6934f2dd62cbe6df11189fb65f,82c5e6b98e498b4150818c913f3e9bc7,delivered,1,74.90,12.65,87.55,2018-02-10,2018-02-10,2018-02-21,2018-02-28,2018-03-14
0b93ff37e8344c6013bad16484c7fbe4,41b40faae6aeea7a8b6884948863386e,delivered,1,998.90,222.38,1221.28,2018-07-22,2018-07-23,2018-08-02,2018-08-07,2018-08-21
b311d7dd519074ae07f219467efbcc6c,958cac30fa94b05f7747e91ed8798ef8,delivered,1,79.90,9.47,89.37,2018-06-10,2018-06-10,2018-06-19,2018-06-20,2018-06-29
f9b9c3c69ef08b6eda32ed086133abb6,2ee87cede5826cd6458393028d3ec739,delivered,1,48.90,9.74,58.64,2018-02-14,2018-02-14,2018-02-16,2018-02-19,2018-03-01
